# Endpoints

In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
df_items = pd.read_parquet('C:\\Users\\fedez\\OneDrive\\Escritorio\\PI-MLOps\\Data\\items.parquet')
df_juegos = pd.read_parquet('C:\\Users\\fedez\\OneDrive\\Escritorio\\PI-MLOps\\Data\\juegos.parquet')
df_resenias = pd.read_parquet('C:\\Users\\fedez\\OneDrive\\Escritorio\\PI-MLOps\\Data\\resenias.parquet')

In [3]:
df_items.sample()

,user_id,items_count,item_id,playtime_forever
3530879,76561197964050011,75,407530,0


In [4]:
df_juegos.sample()

,genres,title,price,id,developer,release_year
20616,simulation,fidget spinner editor - expansion pack 2,0.99,737860,myarcadegames studios,2017


In [5]:
df_resenias.sample()

,user_id,item_id,recommend,analisis_sentimiento
26597,76561198094460843,225540,True,1


In [6]:
# Se cambia el nombre de la columna "id" por "item_id" para poder realizar el merge de los dataframes
df_juegos.rename(columns={'id': 'item_id'}, inplace=True)
df_juegos.sample()

,genres,title,price,item_id,developer,release_year
42089,rpg,valens,7.99,441640,digitallion,2016


### **Función 1**<br>
**def developer( desarrollador : str )**: Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.<br>


### **Función 2**<br>
**def userdata( User_id : str )**: Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.<br>
Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

### **Función 3**<br>
**def UserForGenre( genero : str )**: Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.<br>
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [25]:
df_ju3 = df_juegos[['item_id', 'genres', 'release_year']]
df_it3 = df_items[['item_id', 'user_id','playtime_forever']]
df_filtrado = df_it3[df_it3['playtime_forever'] != 0]
df_1_3 = df_4 = df_ju3.merge(df_filtrado, on='item_id', how='outer')

In [26]:
df_1_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7217835 entries, 0 to 7217834
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   genres            object 
 2   release_year      float64
 3   user_id           object 
 4   playtime_forever  float64
dtypes: float64(2), object(3)
memory usage: 275.3+ MB


In [8]:
def UserForGenre(genero):
    # Se convierte la cadena de texto ingresada en minúscula.
    genero_m = genero.lower()
    
    # Se corrobora que el valor ingresado esté dentro del dataframe.
    df_genero = df_fun_3[df_fun_3["genres"] == genero_m]

        # Se genera una sumatoria de las horas jugadas por año.
    df_horas_anuales = df_genero.groupby(["release_year"])["playtime_forever"].sum()
    df_horas_anuales = df_horas_anuales.reset_index()

    # Se hace una sumatoria de las horas respecto al usuario.
    df_horas = df_genero.groupby("user_id")["playtime_forever"].sum()
    
    # Se calcula el usuario con mayor cantidad de horas acumuladas.
    top_horas = df_horas.idxmax()

    df_horas_anuales = df_horas_anuales.rename(columns={"release_year": "Año", "playtime_forever": "Horas"})
    horas_anuales = df_horas_anuales.to_dict(orient="records")

    return {f"Usuario con más horas jugadas para género {genero}": top_horas, "Horas jugadas": horas_anuales}

### **Función 4**<br>
**def best_developer_year( año : int )**: Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos).<br>
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]



In [31]:
def best_developer_year(anio):

    # Se extraen las columnas de df_resenias necesarias para el merge con df_juegos.
    df_resenias4 = df_resenias[['item_id', 'recommend']]
    df_juegos4 = df_juegos[['item_id', 'developer', 'release_year']]
    df_fun_4 = df_juegos4.merge(df_resenias4, on='item_id', how='outer') 

    # Se corrobora que el input sea correcto.
    if anio not in df_fun_4['release_year'].unique():
        return f"El año {anio} no existe en los registros."
    
    # Filtrar el dataset para obtener solo las filas correspondientes al año dado.
    juegos_del_año = df_fun_4[df_fun_4['release_year'] == anio]

    # Se calculan las reseñas por desarrolladora.
    resenias = juegos_del_año.groupby('developer')['recommend'].sum().reset_index()

    # Se ordenan los juegos mejor valorados en orden ascendente.
    desarrolladoras = resenias.sort_values(by='recommend', ascending=False)

    # Se ordenan los primeros tres puestos
    oro = desarrolladoras.iloc[0]['developer']
    plata = desarrolladoras.iloc[1]['developer']
    bronce = desarrolladoras.iloc[2]['developer']

    # Crear la lista de diccionarios con los tres primeros lugares
    top3 = [{"Puesto 1": oro.title()}, {"Puesto 2": plata.title()}, {"Puesto 3": bronce.title()}]
    return top3

In [44]:
for i in range(2008, 2019):
    print(f'{i} : {best_developer_year(i)}')

2008 : [{'Puesto 1': 'Rockstar New England'}, {'Puesto 2': 'Treyarch'}, {'Puesto 3': 'Cd Projekt Red'}]
2009 : [{'Puesto 1': 'Valve'}, {'Puesto 2': 'Tripwire Interactive'}, {'Puesto 3': 'Dice'}]
2010 : [{'Puesto 1': 'Taleworlds Entertainment'}, {'Puesto 2': 'Obsidian Entertainment'}, {'Puesto 3': 'Avalanche Studios'}]
2011 : [{'Puesto 1': 'Re-Logic'}, {'Puesto 2': 'No More Room In Hell Team'}, {'Puesto 3': 'Valve'}]
2012 : [{'Puesto 1': 'Valve'}, {'Puesto 2': 'Gearbox Software,Aspyr (Mac &Amp; Linux)'}, {'Puesto 3': 'Wild Shadow Studios'}]
2013 : [{'Puesto 1': 'Facepunch Studios'}, {'Puesto 2': 'Bohemia Interactive'}, {'Puesto 3': 'Digital Extremes'}]
2014 : [{'Puesto 1': 'Edge Of Reality'}, {'Puesto 2': 'Endnight Games Ltd'}, {'Puesto 3': 'Coffee Stain Studios'}]
2015 : [{'Puesto 1': 'Psyonix, Inc.'}, {'Puesto 2': 'Kyle Seeley'}, {'Puesto 3': 'Trion Worlds'}]
2016 : [{'Puesto 1': 'Chucklefish'}, {'Puesto 2': 'Reto-Moto'}, {'Puesto 3': 'Doubledutch Games'}]
2017 : [{'Puesto 1': 'Smartl

### **Función 5** <br>
**def developer_reviews_analysis( desarrolladora : str )**: Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.<br>
Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

In [15]:
def developer_reviews_analysis(desarrolladora):

    # Se realiza un merge de las columnas necesarias de "df_juegos" y "df_resenias"
    df_juegos5 = df_juegos[['item_id', 'developer']]
    df_resenias5 = df_resenias[['item_id', 'analisis_sentimiento']]
    df_fun_5 = df_resenias5.merge(df_juegos5, on='item_id', how='outer')

    # Si el valor no es una cadena de texto se devuelve un mensaje de error.
    if type(desarrolladora) != str:
        return 'Error: El valor ingresado debe ser una palabra'
    
    #Se convierten a minúscula los valores ingresados para compatibilizar con los valores de la columna 'developer'.
    desarrollador = desarrolladora.lower()

    #Se corrobora que el valor ingresado se encuentre en la columna.
    desarrollador = df_fun_5[df_fun_5['developer'] == desarrollador]
    
    analisis = desarrollador['analisis_sentimiento']
    # Se hace un conteo de las opiniones.
    opinion = analisis.value_counts()

    return {desarrolladora : list([f'Negative: {(opinion.get(0, 0))}', f'Positive: {(opinion.get(2, 0))}'])}

In [16]:
developer_reviews_analysis('Valve')

{'Valve': ['Negative: 1058', 'Positive: 8028']}